# 🧰 Google Cloud Storage (GCS) Tutorial Notebook
Course: Data Engineering on Google Cloud
Project: Data Analytics using GCS and Pandas

## 📦 Setup and Initialization
✅ Add Google Cloud SDK to PATH

In [15]:
import os 
os.environ["PATH"] += ':/Users/apple/Documents/setups/google-cloud-sdk/bin'

## ✅ Verify gsutil

In [16]:
!which gsutil

/Users/apple/Documents/setups/google-cloud-sdk/bin/gsutil


# 🎯 Creating and Managing GCS Buckets
## 🪣 Create a GCS Bucket

In [17]:
!gsutil mb gs://de_retail_db ## ! this is shell command

Creating gs://de_retail_db/...
ServiceException: 409 A Cloud Storage bucket named 'de_retail_db' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


## 📂 List GCS Buckets

In [19]:
!gsutil ls -r

gs://de_retail_db/


## 📤 Upload Data to GCS

In [20]:
!gsutil cp -r ../data/retail_db gs://de_retail_db/retail_db

Copying file://../data/retail_db/create_db_tables_pg.sql [Content-Type=application/x-sql]...
Copying file://../data/retail_db/schemas.json [Content-Type=application/json]...
Copying file://../data/retail_db/load_db_tables_pg.sql [Content-Type=application/x-sql]...
Copying file://../data/retail_db/customers/part-00000 [Content-Type=application/octet-stream]...
- [4 files][ 10.7 MiB/ 10.7 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../data/retail_db/products/part-00000 [Content-Type=application/octet-stream]...
Copying file://../data/retail_db/departments/part-00000 [Content-Type=application/octet-stream]...
Copying file://../data/retail_db/order_items/part-00000 [Content-Type=application/octet-stream]...
Copying fil

## 📂 Recursively List Bucket Contents

In [21]:
!gsutil ls -r gs://de_retail_db

gs://de_retail_db/retail_db/:
gs://de_retail_db/retail_db/create_db_tables_pg.sql
gs://de_retail_db/retail_db/load_db_tables_pg.sql
gs://de_retail_db/retail_db/schemas.json

gs://de_retail_db/retail_db/categories/:
gs://de_retail_db/retail_db/categories/part-00000

gs://de_retail_db/retail_db/customers/:
gs://de_retail_db/retail_db/customers/part-00000

gs://de_retail_db/retail_db/departments/:
gs://de_retail_db/retail_db/departments/part-00000

gs://de_retail_db/retail_db/order_items/:
gs://de_retail_db/retail_db/order_items/part-00000

gs://de_retail_db/retail_db/orders/:
gs://de_retail_db/retail_db/orders/part-00000

gs://de_retail_db/retail_db/products/:
gs://de_retail_db/retail_db/products/part-00000


# 🔐 Authenticate and Initialize GCS Client
## 🔑 Setup GCP Authentication
```
gcloud auth application-default login
```

## 💻 Create GCS Client

In [22]:
from google.cloud import storage
gsclient = storage.Client()

/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## 📁 Listing Buckets and Blobs
### 🪣 List Buckets

In [23]:
buckets = gsclient.list_buckets()
for bucket in buckets:
    print(bucket.name)

de_retail_db


### 📄 List Blobs in a Bucket

In [24]:
blobs = gsclient.list_blobs('de_retail_db')
for blob in blobs:
    print(blob.name)

retail_db/categories/part-00000
retail_db/create_db_tables_pg.sql
retail_db/customers/part-00000
retail_db/departments/part-00000
retail_db/load_db_tables_pg.sql
retail_db/order_items/part-00000
retail_db/orders/part-00000
retail_db/products/part-00000
retail_db/schemas.json


## 📤 Upload and 📥 Download Files
### Upload a Single File to GCS

In [25]:
bucket = gsclient.get_bucket('de_retail_db')
blob = bucket.blob('pythondemo/retail_db/orders/part-00000') #in-memory
blob.upload_from_filename('../data/retail_db/orders/part-00000')

## Download a File from GCS

In [26]:
blob = bucket.blob('pythondemo/retail_db/orders/part-00000')
with open('orders.csv', 'wb') as orders_csv:
    gsclient.download_blob_to_file(blob, orders_csv)

## 🧹 Clean Up
### Remove Folder from GCS

In [27]:
!gsutil rm -r gs://de_retail_db/pythondemo/

Removing gs://de_retail_db/pythondemo/retail_db/orders/part-00000#1744058480982809...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


# 🧾 Utility: File and Schema Handling
## Get All Relevant Files

In [28]:
import glob
import os

def get_file_names(base_dir):
    files = glob.glob(f'{base_dir}/**', recursive=True)
    return [file for file in files if os.path.isfile(file) and file.endswith('part-00000')]

## 📤 Upload Multiple Files to GCS

In [29]:
files = get_file_names('../data/retail_db')
bucket = gsclient.get_bucket('de_retail_db')
target_base_dir = 'pythondemo'

for file in files:
    print(f'Uploading file {file}')
    file_name = '/'.join(file.split('/')[2:])
    blob = bucket.blob(f'{target_base_dir}/{file_name}')
    blob.upload_from_filename(file)


Uploading file ../data/retail_db/customers/part-00000
Uploading file ../data/retail_db/products/part-00000
Uploading file ../data/retail_db/departments/part-00000
Uploading file ../data/retail_db/order_items/part-00000
Uploading file ../data/retail_db/orders/part-00000
Uploading file ../data/retail_db/categories/part-00000


In [30]:
gsclient.list_blobs('de_retail_db', prefix='pythondemo/') # list of blobs iterator - parameter is bucket name

In [32]:
blobs = gsclient.list_blobs('de_retail_db', prefix='pythondemo/') # list of blobs iterator - parameter is bucket name
for blob in blobs:
    print(blob.name)

pythondemo/retail_db/categories/part-00000
pythondemo/retail_db/customers/part-00000
pythondemo/retail_db/departments/part-00000
pythondemo/retail_db/order_items/part-00000
pythondemo/retail_db/orders/part-00000
pythondemo/retail_db/products/part-00000


# 🐼 Pandas + GCS Integration
## Install Required Libraries


In [33]:
!pip install pandas gcsfs pyarrow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Read CSV from GCS using Pandas

In [34]:
import pandas as pd

df = pd.read_csv('gs://de_retail_db/pythondemo/retail_db/orders/part-00000',
                 sep=',',
                 header=None,
                 names=['order_id', 'order_date', 'order_customer_id', 'order_status'])
df.head()

/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


# 📦 Write to Parquet and Read Back
## Convert CSV to Parquet

In [35]:
df.to_parquet('gs://de_retail_db/retail_db_parquet/orders/part-00000.snappy.parquet', index=False)

## Read Parquet from GCS

In [36]:
df = pd.read_parquet('gs://de_retail_db/retail_db_parquet/orders/part-00000.snappy.parquet')
df.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


## 🔄 Convert and Upload All CSVs as Parquet
### Cleanup

In [37]:
!gsutil rm -r gs://de_retail_db/retail_db_parquet/
!gsutil ls -r gs://de_retail_db/

Removing gs://de_retail_db/retail_db_parquet/orders/part-00000.snappy.parquet#1744059054716379...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
gs://de_retail_db/pythondemo/:

gs://de_retail_db/pythondemo/retail_db/:

gs://de_retail_db/pythondemo/retail_db/categories/:
gs://de_retail_db/pythondemo/retail_db/categories/part-00000

gs://de_retail_db/pythondemo/retail_db/customers/:
gs://de_retail_db/pythondemo/retail_db/customers/part-00000

gs://de_retail_db/pythondemo/retail_db/departments/:
gs://de_retail_db/pythondemo/retail_db/departments/part-00000

gs://de_retail_db/pythondemo/retail_db/order_items/:
gs://de_retail_db/pythondemo/retail_db/order_items/part-00000

gs://de_retail_db/pythondemo/retail_db/orders/:
gs://de_retail_db/pythondemo/retail_db/orders/part-00000

gs://de_retail_db/pythondemo/retail_db/products/:
gs://de_retail_db/pythondemo/retail_db/products/par

## Get Column Names from Schema

In [38]:
import json

def get_columns(schema_file, ds_name):
    with open(schema_file) as f:
        schemas = json.load(f)
    ds_schema = sorted(schemas[ds_name], key=lambda x: x['column_position'])
    return [col['column_name'] for col in ds_schema]

## 🔄 Convert and Upload All CSVs as Parquet

In [39]:
target_base_dir = 'retail_db_parquet'
base_dir = '../data/retail_db'
schemas_file = '../data/retail_db/schemas.json'
bucket = 'de_retail_db'

for file in get_file_names(base_dir):
    file_name = file.split('/')[-2]
    blob_suffix = '/'.join(file.split('/')[-2:])
    columns = get_columns(schemas_file, file_name)
    print(f'Uploading file {file}')
    df = pd.read_csv(file, sep=',', header=None, names=columns)
    df.to_parquet(f'gs://{bucket}/{target_base_dir}/{blob_suffix}.snappy.parquet', index=False)

Uploading file ../data/retail_db/customers/part-00000
Uploading file ../data/retail_db/products/part-00000
Uploading file ../data/retail_db/departments/part-00000
Uploading file ../data/retail_db/order_items/part-00000
Uploading file ../data/retail_db/orders/part-00000
Uploading file ../data/retail_db/categories/part-00000


## ✅ Verify Uploaded Files

In [40]:
!gsutil ls -r gs://de_retail_db/retail_db_parquet/

gs://de_retail_db/retail_db_parquet/:

gs://de_retail_db/retail_db_parquet/categories/:
gs://de_retail_db/retail_db_parquet/categories/part-00000.snappy.parquet

gs://de_retail_db/retail_db_parquet/customers/:
gs://de_retail_db/retail_db_parquet/customers/part-00000.snappy.parquet

gs://de_retail_db/retail_db_parquet/departments/:
gs://de_retail_db/retail_db_parquet/departments/part-00000.snappy.parquet

gs://de_retail_db/retail_db_parquet/order_items/:
gs://de_retail_db/retail_db_parquet/order_items/part-00000.snappy.parquet

gs://de_retail_db/retail_db_parquet/orders/:
gs://de_retail_db/retail_db_parquet/orders/part-00000.snappy.parquet

gs://de_retail_db/retail_db_parquet/products/:
gs://de_retail_db/retail_db_parquet/products/part-00000.snappy.parquet


### Preview Datasets

In [41]:
datasets = ['orders', 'products', 'categories', 'departments', 'customers', 'order_items']
for dataset in datasets:
    df = pd.read_parquet(f'gs://de_retail_db/retail_db_parquet/{dataset}/part-00000.snappy.parquet')
    print(f'{dataset}:')
    print(df.head(2))

orders:
   order_id             order_date  order_customer_id     order_status
0         1  2013-07-25 00:00:00.0              11599           CLOSED
1         2  2013-07-25 00:00:00.0                256  PENDING_PAYMENT
products:
   product_id  product_cateogry_id  \
0           1                    2   
1           2                    2   

                                    product_name  product_description  \
0  Quest Q64 10 FT. x 10 FT. Slant Leg Instant U                  NaN   
1  Under Armour Men's Highlight MC Football Clea                  NaN   

   product_price                                      product_image  
0          59.98  http://images.acmesports.sports/Quest+Q64+10+F...  
1         129.99  http://images.acmesports.sports/Under+Armour+M...  
categories:
   category_id  category_department_id category_name
0            1                       2      Football
1            2                       2        Soccer
departments:
   department_id department_name
0      

## 🚮 Delete File from GCS using Python

In [2]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket('de_retail_db')
blob = bucket.blob('retail_db_parquet/orders/part-00000.snappy.parquet')
blob.delete()
print("File deleted.")

/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


File deleted.


## 🗃️ Upload JSON Data as Object (from memory)

In [3]:
import json
from google.cloud import storage

data = {"key": "value", "status": "active"}

client = storage.Client()
bucket = client.bucket('de_retail_db')
blob = bucket.blob('json_folder/sample.json')
blob.upload_from_string(json.dumps(data), content_type='application/json')

print("JSON object uploaded.")


/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/apple/Documents/GitHub/gcp-cook-book/de_env/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


JSON object uploaded.


## 🎓 Summary
In this notebook, we:

- Set up and authenticated with GCS.

- Created a GCS bucket and uploaded retail data.

- Used gsutil, google.cloud.storage, and pandas to interact with GCS.

- Converted CSVs to Parquet and uploaded them efficiently.

- Read data directly from GCS using Pandas.